In [37]:
from abc import ABC, abstractmethod
import numpy as np
from scipy.io import loadmat
from get_images import get_images

def relu(x, derivative=False):
    if (derivative == True):
        y = np.ones(x.shape)
        y[x <= 0] = 0

        return y
    
    return np.maximum(0, x)


class Layer(ABC):
    @abstractmethod
    def forward(self, x):
        pass
    
    @abstractmethod
    def backward(self, error):
        pass

class Input(Layer):
    def forward(self, x):
        return x

    def backward(self, error):
       return error   

class Activation(Layer):
    def __init__(self, act):
      self.act = act

    def forward(self,x):
      self.capa_anterior = self.act(x)
      self.weight = x
      return self.act(x)
    
    def backward(self, error):
       return error * self.act(self.capa_anterior, derivative=True)
    
# capa anterior = N
class Dense(Layer):    
    def __init__(self, weight, bias):
        self.weight = np.random.normal(0, weight**-0.5, [weight, bias])
        self.bias = np.random.normal(0, weight**-0.5, [1, bias])
    
    def forward(self, input):
        #multicación de matrices el @
        self.bash_size = input.shape[0]
        self.capa_anterior = input
        return input @ self.weight + self.bias

    def update_parameters(self, lr):
       self.weight = self.weight - lr/self.bash_size * self.dW
       self.bias = self.bias - lr/self.bash_size * self.dB


    def backward(self, error):
      self.dW = self.capa_anterior.T @ error
      self.dB = np.sum(error, axis=0, keepdims=True)

      return error @ self.weight.T
      
       
    
class Model():
    def __init__(self, layers):
      self.layers = layers

    def forward(self, x):
        self.batch_size = x.shape[0]
        output = x
        for layer in self.layers:
           output = layer.forward(output)

        return output
    
    def backward(self, y_test, y_pred):
        error = self.cost_function(y_test, y_pred)

        for layer in reversed(self.layers):
            error = layer.backward(error)
        return error
    
    def update_parameters(self, lr):
      for layer in self.layers:
          if isinstance(layer, Dense):
            layer.update_parameters(lr)

    def cost_function(self, y_test, y_pred, derivative=False):
      if derivative == False:
        return np.square(y_test - y_pred)
      
      return (y_test - y_pred) * 2

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) 

def train(x, y , my_first_NN, lr=0.01):
    y_pred = my_first_NN.forward(x)
    
    my_first_NN.backward(y_pred, y)
    my_first_NN.update_parameters(lr)
    
    return my_first_NN.cost_function(y_pred, y)

def test(x, y, my_first_NN):
    y_pred = my_first_NN.forward(x)
    print("Acc: ", accuracy(y_pred, y))
    
    return accuracy(y_pred, y)

def accuracy(y_pred, y):
    return np.argmax(y_pred, axis=1) == np.argmax(y, axis=1) / len(y)


In [34]:
def load_mnist():
  mnist_path = './datasets/'
  x_train, y_train, x_test, y_test = get_images(mnist_path)
  x_train = x_train[:60000].reshape(60000, -1).astype(np.float32)/255
  y_train = y_train[:60000].reshape(60000, 1)

  x_test = x_test.copy().reshape(10000, -1)
  y_test = y_test.copy().reshape(10000, 1).astype(np.float32)/255

  
  return x_train, y_train, x_test, y_test


def split_dataset(x_data, y_data, batch_size):
  data = []

  for i in range(0, len(x_data), batch_size):
    x = x_data[i:i+batch_size]
    y = y_data[i:i+batch_size]
    data.append((x, y))

  return data


In [35]:
def save_model(model, path):
  np.save(path, model)

In [39]:
# ## main things


def main():
  batch_size = 500
  epochs = 10
  curr_epoch = 0
  ninputs = 784
  hl1 = 1000
  hl2 = 100
  nclass = 10
  lr = 0.03

  # # Crear modelo
  input = Input()
  layer_relu = Activation(relu)
  layer_relu2 = Activation(relu)
  dense1 = Dense(ninputs, hl1)
  dense2 = Dense(hl1, hl2)
  dense3 = Dense(hl2, nclass)

  NN = Model([input, dense1, layer_relu, dense2, layer_relu2, dense3])


  x_train, y_train, x_test, y_test = load_mnist()

  train_data = split_dataset(x_train, y_train, batch_size)
  test_data = split_dataset(x_test, y_test, batch_size)

  # while curr_epoch < epochs:
    # for i, batch in enumerate(train_data):
      # x, y = batch
  x, y = train_data[0]
  cost = train(x, y, NN, lr)
  
    #   cost = train(x, y, NN)
    #   print(cost)
    # acuracity = test(test_data, NN)
    # if acuracity > 0.97:
    #   save_model()
    #   break;
    
    # epoch += 1
  

main()


[ 0.06560913  0.07312453  0.09119536  0.20601443  0.14483701 -0.07317141
  0.04009404 -0.12444962 -0.18507495  0.24494263]
